Firstly we will import all the dependencies 

In [1]:
#!pip install scikit-learn
#!pip install tensorflow

In [4]:
import json
import os
import shutil
import cv2
import numpy as np
import os
# from matplotlib import pyplot as plt
# import time
import mediapipe as mp
# import re

# import cv2
# import numpy as np
# import os

2023-12-30 10:52:20.707698: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-30 10:52:20.741705: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 10:52:20.741762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 10:52:20.742684: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 10:52:20.748338: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
video_data_folder = "../VideoData"
bds_folder = "../BdSLW60"
numerical_data_path='../NumericalData'

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
# mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
# def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
#     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
#     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
# def draw_styled_landmarks(image, results):
#     # Draw face connections
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
#                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
#                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
#                              )


# 3. Extract Keypoint Values


In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3) 
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [8]:
# A NumPy array with all values set to 0


In [9]:
# def is_numeric(s):
#     try:
#         float(s)
#         return True
#     except ValueError:
#         return False

In [36]:
# Step 2: Open the JSON file
datas = []
word_lists = os.listdir(bds_folder)
for word in word_lists:
    if word.endswith('.pdf') or word.endswith('.xlsx') or word.endswith('.txt'):
        pass 
    else:
        with open(f'{bds_folder}/{word}/output1.json', 'r') as file:
            # Step 3: Parse the JSON data into a Python data structure
            datas.append(json.load(file))


In [11]:
# import json
# #Read JSON files

# JSON_PATH = '../Preprocessing/output1.json'

# JSON_file = open(JSON_PATH)

# JSON_DATA = json.load(JSON_file)

In [13]:


# Create a folder named 'VideoData' in the code directory

os.makedirs(video_data_folder, exist_ok=True)

# Path to the folder containing your videos


# Iterate through each subfolder in the BdSLW60 folder
for subfolder_name in os.listdir(bds_folder):
    subfolder_path = os.path.join(bds_folder, subfolder_name)

    # Check if the item in the BdSLW60 folder is a directory
    if os.path.isdir(subfolder_path):
        # Iterate through video files in the current subfolder
        for video_file_name in os.listdir(subfolder_path):
            video_file_path = os.path.join(subfolder_path, video_file_name)

            # Check if the item is a file and ends with ".mp4"
            if os.path.isfile(video_file_path) and video_file_name.endswith(".mp4"):
                # Extract the word from the filename (assuming it's always the second character)
                w_idx = 0

                f_idx = 0
                for idx in range(0,len(video_file_name)):
                    if video_file_name[idx]=='W':
                        w_idx = idx 
                        break
                
                for idx in range(0,len(video_file_name)):
                    if video_file_name[idx]=='F':
                        f_idx = idx 
                        break
                word = video_file_name[w_idx+1:f_idx]
                # Create a corresponding folder in 'VideoData' based on the word
                video_data_subfolder = os.path.join(video_data_folder, f"W{word}")
                os.makedirs(video_data_subfolder, exist_ok=True)

                # Copy the video file to the corresponding folder in 'VideoData'
                shutil.copy(video_file_path, os.path.join(video_data_subfolder, video_file_name))

print("Video files copied successfully.")

Video files copied successfully.


In [14]:


# Define the folder containing the video files
#VIDEO_FOLDER = 'VideoData'
#all_words = os.listdir(VIDEO_FOLDER)
# Actions that we try to detect
#actions = np.array(all_words)

# Function to process a video file
def process_video(video_path, fileName_path):
    cap = cv2.VideoCapture(video_path)
    # print(video_path)
    sequence_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sequence = 1
    # Set mediapipe model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                if not ret:
                    break

                # Make de8tections
                image, results = mediapipe_detection(frame, holistic)
               
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(fileName_path, str(sequence))
                np.save(npy_path, keypoints)
                sequence += 1
                # Break gracefully
                #if cv2.waitKey(10) & 0xFF == ord('q'):
                #    break
    
    cap.release()
    cv2.destroyAllWindows()



In [15]:
#datas

In [16]:
def extractNpy(source_folder, fileName_path, fileName):
    video_path = os.path.join(source_folder, fileName+'.mp4')
    print(video_path)
    process_video(video_path, fileName_path)
    print('Npy creation finished for file: '+fileName)
    # sequence_path = os.path.join(f'Data/{action}',fileName)
    # cam = cv2.VideoCapture(video_path)
    # fps = cam.get(cv2.CAP_PROP_FPS)


In [38]:
import math
max_frame = 0
min_frame = 99999980
user = ''

max_start = 0
max_end = 0
max_ind = 0


min_start = 0
min_end = 0
min_ind = 0

diff = 0

total_frameCount= 0
total_trails=0

training_left_trials=0
training_right_trials=0

test_right_trials =0
test_left_trials=0


testUser = {'U4','U8'}

In [39]:
for data in datas:
    for word in data:
        for user in data[word]:
            for orientation in data[word][user]:
                 for fileName in data[word][user][orientation]:
                    
                    frameRate = data[word][user][orientation][fileName]['FrameRate']
                    no_trials = len(data[word][user][orientation][fileName]['trials'])
                    
                    #override the trial count as sometime that is missing in annotation
                    data[word][user][orientation][fileName]['no_of_trials']= no_trials

                    if user+word+'F' in fileName:
                        pass 
                    else:
                        print(fileName,user+word+'F',' file annotation err')
                    if orientation == 'RightHand' or orientation == 'LeftHand':
                        pass
                    else:
                        print(fileName,user+word+'F',' orientation annotation err')
                    
                    if frameRate == '30' or frameRate == '24' or frameRate == '15':
                        pass
                    else:
                        print (fileName,' annotation frame rate error')

                    data_path = f'{numerical_data_path}/'
                    if user in testUser:
                        data_path = data_path+'Test/'
                    else:
                        data_path = data_path+'Training/'
            
                    data_path = data_path + orientation
                    #if orientation == 'RightHand' or orientation == 'LeftHand':
                        # Orientation Folder Creation
                        #if os.path.exists(data_path):
                        #    pass
                        #else:
                        #    os.makedirs(data_path)
                            
                    # Word Folder Creation
                    word_path = data_path+'/'+word
                    
                    #if os.path.exists(word_path):
                    #    pass
                    #else:
                    #    os.makedirs(word_path)
                        
                    # User Folder Creation
                    user_path = word_path+'/'+user
                
                    #if os.path.exists(user_path):
                    #    pass
                    #else:
                    #    os.makedirs(user_path)
                     
                    fileName_path = user_path+'/'+fileName
                    #if os.path.exists(fileName_path):
                    #    pass
                    #else:
                    #    os.makedirs(fileName_path)
                        
                
                    all_npy_path = fileName_path+'/'+'all_npy'
                
                    source_path = f'{video_data_folder}/{word}/'
                    #print("Extracting npy ",source_path,fileName_path)
                    exception_flag = False
                    already_npy = False
                    try:
                        if os.path.exists(all_npy_path):
                            pass
                            already_npy = True
                            #print('already npy')
                        else: 
                            os.makedirs(all_npy_path)
                            # print("all npy created")
                            extractNpy(source_path,all_npy_path,fileName)
                            already_npy = False
                    except:
                        exception_flag = True
                        print('Something Wrong in the file : ',all_npy_path)
                    #Trial Folders Creation
                    trial_path = fileName_path+'/'
                
                
                    i=0
                    for i in range(0,no_trials):
                        trial_folder = trial_path+f't{str(i)}'
                        
                        if os.path.exists(trial_folder):
                            pass
                            
                        else:
                            os.makedirs(trial_folder)
                            
                            
                            #print(f'{trial_folder} folder created.')
                        starting = data[word][user][orientation][fileName]['trials'][str(i)]['starting']
                        ending = data[word][user][orientation][fileName]['trials'][str(i)]['ending']
                        diff = ending - starting
                        if frameRate == '15':
                            diff = diff * 2
                        elif frameRate == '24':
                            diff = math.ceil(diff * (5/4))
                        #elif frameRate == '30':
                        #    diff = diff
                        # else:
                        #       print("Error in Frame Rate in Annotation :",frameRate)
                        
                        if exception_flag==False and already_npy ==False:
                            frame=0
                            for frame in range(starting,ending+1):
                                #print(word,user,i,frame)
                                shutil.copy(all_npy_path+'/'+str(frame)+'.npy', trial_folder+'/'+str(frame)+'.npy')
                        #else:
                        #    print(f'Orientation Error for file {word} {user}')
                        
                        # do some statistics-----------------------------
                        total_trails +=1

                        if user in testUser:
                            if orientation == 'RightHand':
                                test_right_trials+=1
                            elif orientation == 'LeftHand':
                                test_left_trials+=1
                        else:
                            if orientation == 'RightHand':
                                training_right_trials+=1
                            elif orientation == 'LeftHand':
                                training_left_trials+=1

                        total_frameCount+=diff
                        

                        if diff > max_frame: 
                            max_frame = diff
                            max_start = starting
                            max_end = ending   
                            max_filename = fileName
                            max_ind = i 
                        if diff < min_frame: 
                            min_frame = diff
                            min_start = starting
                            min_end = ending    
                            min_filename = fileName
                            min_ind = i 
                        # statistics end ----------------------------------

                    # end of trial loop
                    
MAX_FRAME =max_frame+1
min_frame=min_frame+1
# total_frameCount +=total_trails
avegra_no_frames= 1+total_frameCount/total_trails
print("Max frame is",MAX_FRAME,max_filename,max_ind,max_start, max_end)
print("Min frame is",min_frame,min_filename,min_ind,min_start, min_end)
print("No of trials: ",total_trails)
print("Avg frames per trial: ",avegra_no_frames)
print('Total Training instances: ',training_right_trials+training_left_trials,' Right Hand: ',training_right_trials,' Left Hand: ',training_left_trials)
print('Total Test instances: ',test_right_trials+test_left_trials,' Right Hand: ',test_right_trials,' Left Hand: ',test_left_trials)
# print('TOTAL INSTANCES: ',training_right_trials+training_left_trials+test_right_trials+test_left_trials)


Max frame is 164 U17W355F 5 878 1041
Min frame is 9 U8W216F 10 772 780
No of trials:  9307
Avg frames per trial:  44.20027935962179
Total Training instances:  8031  Right Hand:  6530  Left Hand:  1501
Total Test instances:  1276  Right Hand:  1143  Left Hand:  133


In [33]:

for data in datas:
    for word in data:
        for user in data[word]:
            for orientation in data[word][user]:
                 for fileName in data[word][user][orientation]:
                    
                    frameRate = data[word][user][orientation][fileName]['FrameRate']
                    no_trials = len(data[word][user][orientation][fileName]['trials'])
                    
                    #override the trial count as sometime that is missing in annotation
                    data[word][user][orientation][fileName]['no_of_trials']= no_trials

                    #fileName_text = data[word][user][orientation][fileName]['FileName']
                    #orientation_text = data[word][user][orientation][fileName]['Orientation']
                    
                    data_path = f'{numerical_data_path}/'
                    destination_path = data_path+'/Calibrated'
                    if user in  testUser:
                        data_path = data_path+'/Test'
                        destination_path = destination_path+'/Test'
                    else:
                        data_path = data_path+'/Training'
                        destination_path = destination_path+'/Training'

                    lhFlag = False
                    if(orientation == 'LeftHand'):
                        lhFlag = True
                        # flip_destination_path = f'{destination_path}/{orientation}_FLIPPED/{word}/{user}/{fileName}' 
                        flip_cal_destination_path = f'{destination_path}/{orientation}_FLIPPED_CALIB/{word}/{user}/{fileName}' 
                        #print(flip_destination_path)
                    #print(flip_destination_path)
                    #print(lhFlag)
                    data_path = f'{data_path}/{orientation}/{word}/{user}/{fileName}'
                    destination_path = f'{destination_path}/{orientation}/{word}/{user}/{fileName}'                          
                    
                    
                    

                    ref_x=0
                    ref_y=0
                    ref_d=0
                    i=0
                    for i in range(0,no_trials):
                        src_trial_folder = f'{data_path}/t{i}'
                        dest_trial_folder = f'{destination_path}/t{i}'
                        if os.path.exists(dest_trial_folder):
                            pass
                        else:
                            os.makedirs(dest_trial_folder)
                        #print(lhFlag)
                        if(lhFlag == True):
                            #print('left hand trial folder creation if')
                            # flip_dest_trial_folder = f'{flip_destination_path}/t{i}'
                            flip_cal_dest_trial_folder = f'{flip_cal_destination_path}/t{i}'
                            # if os.path.exists(flip_dest_trial_folder):
                                
                            #     pass
                            # else:
                            #     os.makedirs(flip_dest_trial_folder)
                                

                            if os.path.exists(flip_cal_dest_trial_folder):
                                #print(flip_dest_trial_folder,' exists')
                                pass
                            else:
                                os.makedirs(flip_cal_dest_trial_folder)    
                            
                        starting = data[word][user][orientation][fileName]['trials'][str(i)]['starting']
                        ending = data[word][user][orientation][fileName]['trials'][str(i)]['ending']
                        # if first trial, starting frame will be calibration frame
                        
                        if i == 0:
                            #load the starting npy
                            starting_npy=f'{src_trial_folder}/{starting}.npy'
                            #print(starting_npy)
                            calib_npy=np.load(starting_npy);
                            #print(calib_npy)
                            #pose 
                            #face 
                            #lh 
                            #rh 
                            keypoints3D=calib_npy.reshape(-1,3)
                            left_shoulder = keypoints3D[11]
                            right_shoulder = keypoints3D[12]
                            ref_x=(left_shoulder[0]+right_shoulder[0])/2
                            ref_y=(left_shoulder[1]+right_shoulder[1])/2
                            ref_d=(left_shoulder[2]+right_shoulder[2])/2
                            #print('start: ')
                            #print(ref_x,left_shoulder[0],right_shoulder[0],ref_y,left_shoulder[1],right_shoulder[1],ref_d,left_shoulder[2],right_shoulder[2] )
                            #print(ref_x,left_shoulder[0]] )
                        
                        #calibrate all frames, FLIP left hand with right hand, calibrate flipped
                        frame=0
                        for frame in range(starting,ending+1):
                            src_npy_path=f'{src_trial_folder}/{frame}.npy'
                            src_npy=np.load(src_npy_path);
                            srcKeypoints3D=src_npy.reshape(-1,3)

                            if(len(srcKeypoints3D) ==543 ):
                                pass
                            else:
                                print('LandMark error during npy')
                            
                            if (lhFlag == True):
                                flip_src_npy=np.load(src_npy_path);
                                flip_srcKeypoints3D=flip_src_npy.reshape(-1,3)
                                flip_ref_x = (flip_srcKeypoints3D[11][0]+flip_srcKeypoints3D[12][0])/2
                            #print(srcKeypoints3D)
                            #normalize the keypoints
                            #print(srcKeypoints3D[1628])
                            
                            #print(ref_x)
                            
                            
                            #print(ref_y)
                            #calibratedNp=np.array={}
                            #deal with missing values also........
                            keypoint_i=0
                            for keypoint_i in range(0,543):

                                src3d=srcKeypoints3D[keypoint_i]
                                x=src3d[0]
                                y=src3d[1]
                                d=src3d[2]
                                if(x<=0.0 and y <=0.0 and d<=0.0) :
                                   #missing poing
                                   continue

                                src3d[0]=x-ref_x
                                src3d[1]=y-ref_y
                                src3d[2]=d-ref_d

                                if(lhFlag == True):
                                    #flip on original data only hand points last 21 and then 21 points
                                    # swapping not done yet
                                    mirror_points={13,14,15,16,17,18,19.20,21,22} 
                                    if(keypoint_i in mirror_points or keypoint_i >=501):
                                        flip_src3d=flip_srcKeypoints3D[keypoint_i]
                                        flip_src3d[0]=flip_src3d[0]-flip_ref_x
                                        flip_src3d[0]=-flip_src3d[0]
                                        flip_src3d[0]=flip_ref_x+flip_src3d[0] 
                                                               
                            #for finished        
                                
                                
                                

                            dest_trial_npy_path = dest_trial_folder+'/'+str(frame)+'.npy'
                            np.save(dest_trial_npy_path,srcKeypoints3D.flatten())
                            # exchange hands and shoulder point for LeftHand Orientation
                            if (lhFlag == True):
                                #swap hands and related pose points

                                swap_points={11,13,15,17,19,21} 
                                sp=0
                                for sp in swap_points:
                                    left_x= flip_srcKeypoints3D[sp][0]
                                    left_y= flip_srcKeypoints3D[sp][1]
                                    left_d= flip_srcKeypoints3D[sp][2]
                                
                                    right_x= flip_srcKeypoints3D[sp+1][0]
                                    right_y= flip_srcKeypoints3D[sp+1][1]
                                    right_d= flip_srcKeypoints3D[sp+1][2]

                                                               
                                    flip_srcKeypoints3D[sp][0] =right_x
                                    flip_srcKeypoints3D[sp][1] =right_y
                                    flip_srcKeypoints3D[sp][2] =right_d
                                
                                    flip_srcKeypoints3D[sp+1][0]= left_x
                                    flip_srcKeypoints3D[sp+1][1]= left_y
                                    flip_srcKeypoints3D[sp+1][2]= left_d

                                #left right hand swap
                                hpoint_i=0
                                for hpoint_i in range(0,21):
                                    left_x=flip_srcKeypoints3D[501+hpoint_i][0]
                                    left_y=flip_srcKeypoints3D[501+hpoint_i][1]
                                    left_d=flip_srcKeypoints3D[501+hpoint_i][2]
                                    
                                    right_x=flip_srcKeypoints3D[522+hpoint_i][0]
                                    right_y=flip_srcKeypoints3D[522+hpoint_i][1]
                                    right_d=flip_srcKeypoints3D[522+hpoint_i][2]

                                    flip_srcKeypoints3D[501+hpoint_i][0] =right_x
                                    flip_srcKeypoints3D[501+hpoint_i][1] =right_y
                                    flip_srcKeypoints3D[501+hpoint_i][2] =right_d

                                    flip_srcKeypoints3D[522+hpoint_i][0] =left_x
                                    flip_srcKeypoints3D[522+hpoint_i][1] =left_y
                                    flip_srcKeypoints3D[522+hpoint_i][2] =left_d
                                                                        
                                    
                                    

                                #flip_dest_trial_npy_path = flip_dest_trial_folder+'/'+str(frame)+'.npy'
                                

                                # np.save(flip_dest_trial_npy_path,flip_srcKeypoints3D.flatten())
                                
                                
                                #lefthand flip without calibration finished

                                # do calibration for this frame
                                handkey_i=0
                                for handkey_i in range(0,543):
                                    flip_src3d=flip_srcKeypoints3D[handkey_i]
                                    flip_x=flip_src3d[0]
                                    flip_y=flip_src3d[1]
                                    flip_d=flip_src3d[2]
                                    #missing value
                                    if(flip_x<=0.0 and flip_y <=0.0 and flip_d <=0.0):
                                        continue
                                    flip_src3d[0]=flip_x-ref_x
                                    flip_src3d[1]=flip_y-ref_y
                                    flip_src3d[2]=flip_d-ref_d
                                flip_cal_dest_trial_npy_path = flip_cal_dest_trial_folder+'/'+str(frame)+'.npy'
                                #print(flip_dest_trial_npy_path)
                                np.save(flip_cal_dest_trial_npy_path,flip_srcKeypoints3D.flatten())
                            #if lhFlag True did flip and calibrations
                            #shutil.copy(src_trial_folder+'/'+str(frame)+'.npy', dest_trial_folder+'/'+str(frame)+'.npy')
                        # loop for frames of a trial
                    #loop for trials
                    
                        
                        



In [20]:
#from sklearn.model_selection import train_test_split
#from tensorflow.keras.utils import to_categorical

In [21]:
def insertionSort(array):

    for step in range(1, len(array)):
        key = array[step]
        j = step - 1
        
        # Compare key with each element on the left of it until an element smaller than it is found
        # For descending order, change key<array[j] to key>array[j].        
        while j >= 0 and key < array[j]:
            array[j + 1] = array[j]
            j = j - 1
        
        # Place key at after the element just smaller than it.
        array[j + 1] = key

In [22]:
def my_flatten(xss):
    return[x for xs in xss for x in xs]

In [23]:
# generating index for class labels
# the index will be used as label in ML

# class one hot encoding
classAnnotation={}
for data in datas:
    for word in data:
        try:
            classAnnotation[word]        
        except:
            classAnnotation[word]={}
        try:
            classAnnotation[word]['ClassLabel']        
        except:
            classAnnotation[word]['ClassLabel'] ={}
        classAnnotation[word]['ClassLabel'] =word

print(len(classAnnotation))
print(classAnnotation)

numbers = []
for cl in classAnnotation:
    classLabel=classAnnotation[cl]['ClassLabel']
    temp = int(classLabel[1:])
    numbers.append(temp)
insertionSort(numbers)
print(numbers)

classIndex=0
for num in numbers:
    classLabel='W'+str(num)
    try:
        classAnnotation[classLabel]['ClassNumer']
    except:
        classAnnotation[classLabel]['ClassNumer']={}
    classAnnotation[classLabel]['ClassNumer'] =classIndex
    classIndex =classIndex+1

NO_CLASSES =len(classAnnotation)
print(classIndex)
print(classAnnotation)


60
{'W355': {'ClassLabel': 'W355'}, 'W356': {'ClassLabel': 'W356'}, 'W215': {'ClassLabel': 'W215'}, 'W216': {'ClassLabel': 'W216'}, 'W19': {'ClassLabel': 'W19'}, 'W20': {'ClassLabel': 'W20'}, 'W213': {'ClassLabel': 'W213'}, 'W214': {'ClassLabel': 'W214'}, 'W91': {'ClassLabel': 'W91'}, 'W92': {'ClassLabel': 'W92'}, 'W357': {'ClassLabel': 'W357'}, 'W358': {'ClassLabel': 'W358'}, 'W3': {'ClassLabel': 'W3'}, 'W4': {'ClassLabel': 'W4'}, 'W351': {'ClassLabel': 'W351'}, 'W352': {'ClassLabel': 'W352'}, 'W11': {'ClassLabel': 'W11'}, 'W12': {'ClassLabel': 'W12'}, 'W359': {'ClassLabel': 'W359'}, 'W360': {'ClassLabel': 'W360'}, 'W37': {'ClassLabel': 'W37'}, 'W38': {'ClassLabel': 'W38'}, 'W99': {'ClassLabel': 'W99'}, 'W100': {'ClassLabel': 'W100'}, 'W49': {'ClassLabel': 'W49'}, 'W50': {'ClassLabel': 'W50'}, 'W43': {'ClassLabel': 'W43'}, 'W44': {'ClassLabel': 'W44'}, 'W47': {'ClassLabel': 'W47'}, 'W48': {'ClassLabel': 'W48'}, 'W1': {'ClassLabel': 'W1'}, 'W2': {'ClassLabel': 'W2'}, 'W93': {'ClassLabe

In [24]:
LEFT_HAND_ORIENTATION ='LeftHand'

In [25]:
#generating npy array for ML




for data in datas:
    for word in data:
        for user in data[word]:
            for orientation in data[word][user]:
                 
                 
                 for fileName in data[word][user][orientation]:
                    
                    frameRate = data[word][user][orientation][fileName]['FrameRate']
                    no_trials = len(data[word][user][orientation][fileName]['trials'])
                    
                    #override the trial count as sometime that is missing in annotation
                    data[word][user][orientation][fileName]['no_of_trials']= no_trials

                    if frameRate == '30' or frameRate == '24' or frameRate == '15':
                        pass
                    else:
                        print (fileName,' frame rate error in NPY FRAME CONVERSION')

                    
                    
                    #data_path = 'NumericalData'
                    #data_path = data_path+'/Calibrated'

                    trainorTest = 'Training'
                    if user in testUser:
                        trainorTest='Test'
                    
                    
                    ori_folder = orientation
                    if orientation == LEFT_HAND_ORIENTATION:
                        data_path_org_left = f'{numerical_data_path}/Calibrated/{trainorTest}/{ori_folder}/{word}/{user}/{fileName}'
                        dest_path_org_left = f'{numerical_data_path}/ML/{trainorTest}/{orientation}_NO_FLIP'
                        ori_folder ='LeftHand_FLIPPED_CALIB'
                    
                    data_path = f'{numerical_data_path}/Calibrated/{trainorTest}/{ori_folder}/{word}/{user}/{fileName}'

                    dest_path = f'{numerical_data_path}/ML/{trainorTest}/{orientation}'
                    #classLabel = word
                    classIndex =classAnnotation[word]['ClassNumer']
                    #print(classLabel,' ',classIndex)

                   
                    
                    #frame_dup_counter= frameDupRate
                    trial=0
                    for trial in range(0,no_trials):
                   
                        trial_path=f'{data_path}/t{trial}'
                        if orientation == 'LeftHand':
                            trial_path_org_left=f'{data_path_org_left}/t{trial}'

                        #print(trial_path,' ',classIndex)
                        starting = data[word][user][orientation][fileName]['trials'][str(trial)]['starting']
                        ending = data[word][user][orientation][fileName]['trials'][str(trial)]['ending']
                        npy_array=[]
                        if orientation == LEFT_HAND_ORIENTATION:
                            npy_array_org_left=[]
                        
                        fcount=0
                        dupcount=0
                        npy_frame=0
                        for npy_frame in range(starting,ending+1):
                            fcount=fcount+1;
                            if (fcount+dupcount) > MAX_FRAME:
                                print('Frame Excceded')
                                continue

                            npy_frame_path =f'{trial_path}/{npy_frame}.npy'
                                                           
                            npy=np.load(npy_frame_path)
                            #print(len(npy))
                            if (len(npy) >1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK')
                            npy_array.append(npy)
                            

                            if orientation == LEFT_HAND_ORIENTATION:
                                npy_frame_path_org_left =f'{trial_path_org_left}/{npy_frame}.npy'
                                npy_org_left=np.load(npy_frame_path_org_left)
                                npy_array_org_left.append(npy_org_left)
                                

                            #npy_array_oneIns[fcount]=npy
                            #npy.reshape(-1,3)
                            
                            

                            
                            
                            if frameRate =='15':
                                #npy dup
                                npy_array.append(npy)
                                
                                if orientation == LEFT_HAND_ORIENTATION:
                                    npy_array_org_left.append(npy_org_left)
                                    
                                dupcount =dupcount+1
                                
                            elif frameRate == '24' and fcount % 4 == 0:
                                #npy dup
                                npy_array.append(npy)
                                
                                if orientation == LEFT_HAND_ORIENTATION:
                                    npy_array_org_left.append(npy_org_left)
                                    
                                dupcount =dupcount+1
                        #for srtaring ending frame of a trial
                       
                        
                        fcount =fcount+dupcount
                        #print('count: ',count)
                        count=0
                        for count in range(fcount,MAX_FRAME):
                            npy =np.zeros(543*3)
                            npy_array.append(npy)
                            
                            if orientation == LEFT_HAND_ORIENTATION:
                                npy =np.zeros(543*3)
                                npy_array_org_left.append(npy)
                        #print('end count: ',count)    
                        #now class index
                        #class_array.append(classIndex);
                        #print(len(npy_array))
                        dest_class_path=f'{dest_path}/{str(classIndex)}'
                        if os.path.exists(dest_class_path):
                            pass
                        else:
                            os.makedirs(dest_class_path)
                        
                        dest_file_path =f'{dest_class_path}/{classIndex}_{user}_{trial}_{fileName}.npy'                                            
                        np.save(dest_file_path,my_flatten(npy_array))

                        if orientation == LEFT_HAND_ORIENTATION: 
                            dest_class_path_org_left=f'{dest_path_org_left}/{str(classIndex)}'
                            if os.path.exists(dest_class_path_org_left):
                                pass
                            else:
                                os.makedirs(dest_class_path_org_left)
                            
                            dest_file_path_org_left =f'{dest_class_path_org_left}/{classIndex}_{user}_{trial}_{fileName}.npy'                        
                            np.save(dest_file_path_org_left,my_flatten(npy_array_org_left))


                        

                        
                            


                            

                            
                            
                            




In [ ]:
#generating npy array for ML 
#PROLONG THE SHORT GESTURE BY DUPLICATING




for data in datas:
    for word in data:
        for user in data[word]:
            for orientation in data[word][user]:
                 
                 
                 for fileName in data[word][user][orientation]:
                    
                    frameRate = data[word][user][orientation][fileName]['FrameRate']
                    no_trials = len(data[word][user][orientation][fileName]['trials'])
                    
                    #override the trial count as sometime that is missing in annotation
                    data[word][user][orientation][fileName]['no_of_trials']= no_trials

                    if frameRate == '30' or frameRate == '24' or frameRate == '15':
                        pass
                    else:
                        print (fileName,' frame rate error in NPY FRAME CONVERSION')

                    if (len(npy) >1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK')
                    if (len(npy) <1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK',fileName)
                    #data_path = 'NumericalData'
                    #data_path = data_path+'/Calibrated'

                    trainorTest = 'Training'
                    if user in testUser:
                        trainorTest='Test'
                    
                    
                    ori_folder = orientation
                    if orientation == LEFT_HAND_ORIENTATION:
                        data_path_org_left = f'{numerical_data_path}/Calibrated/{trainorTest}/{ori_folder}/{word}/{user}/{fileName}'
                        dest_path_org_left = f'{numerical_data_path}/ML_PROLONG/{trainorTest}/{orientation}_NO_FLIP'
                        ori_folder ='LeftHand_FLIPPED_CALIB'
                    
                    data_path = f'{numerical_data_path}/Calibrated/{trainorTest}/{ori_folder}/{word}/{user}/{fileName}'

                    dest_path = f'{numerical_data_path}/ML_PROLONG/{trainorTest}/{orientation}'
                    #classLabel = word
                    classIndex =classAnnotation[word]['ClassNumer']
                    #print(classLabel,' ',classIndex)

                   
                    
                    #frame_dup_counter= frameDupRate
                    trial=0
                    for trial in range(0,no_trials):
                   
                        trial_path=f'{data_path}/t{trial}'
                        if orientation == 'LeftHand':
                            trial_path_org_left=f'{data_path_org_left}/t{trial}'

                        #print(trial_path,' ',classIndex)
                        starting = data[word][user][orientation][fileName]['trials'][str(trial)]['starting']
                        ending = data[word][user][orientation][fileName]['trials'][str(trial)]['ending']
                        npy_array=[]
                        if orientation == LEFT_HAND_ORIENTATION:
                            npy_array_org_left=[]
                        
                        
                        tf =math.ceil((ending-starting+1)*30/int(frameRate))
                        dupRatio =MAX_FRAME//tf
                        
                        fcount=0
                        dupcount=0
                        spreadCount=0
                        npy_frame=0
                        for npy_frame in range(starting,ending+1):
                            fcount=fcount+1;
                            if (fcount+dupcount) > MAX_FRAME:
                                print('Frame Excceded')
                                continue

                            npy_frame_path =f'{trial_path}/{npy_frame}.npy'
                                                           
                            npy=np.load(npy_frame_path)
                            #print(len(npy))
                            if (len(npy) >1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK')
                            if (len(npy) <1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK',fileName)
                            
                            
                            npy_array.append(npy)
                            #print('DUP ratio: ',dupRatio)
                            dup=1;
                            for dup in range(1,dupRatio):
                                # print ('duplicate times: ',dup, trial,word,user,dupRatio)
                                 npy_array.append(npy)
                                 dupcount=dupcount+1
                            
                            #uniformly spread/duplicate the frames upto max frame
                            scFlag=False
                            if(spreadCount<(MAX_FRAME -tf*dupRatio)):
                                 #print('spread: ',trial,word,user)
                                 npy_array.append(npy)
                                 spreadCount =spreadCount+1
                                 scFlag=True
                                 dupcount=dupcount+1
                                
                            

                            if orientation == LEFT_HAND_ORIENTATION:
                                npy_frame_path_org_left =f'{trial_path_org_left}/{npy_frame}.npy'
                                npy_org_left=np.load(npy_frame_path_org_left)
                                npy_array_org_left.append(npy_org_left)
                                
                                dup=1
                                for dup in range(1,dupRatio):
                                    npy_array_org_left.append(npy_org_left)
                                #uniformly spread/duplicate the frames upto max frame
                                
                                if scFlag==True:
                                    npy_array_org_left.append(npy_org_left)
                                 
                                

                            #npy_array_oneIns[fcount]=npy
                            #npy.reshape(-1,3)
                            
                            

                            
                            
                            if frameRate =='15':
                                #npy dup
                                npy_array.append(npy)
                                dupcount =dupcount+1
                                
                                dup=1
                                for dup in range(1,dupRatio):
                                    npy_array.append(npy)
                                    dupcount=dupcount+1
                                
                                if orientation == LEFT_HAND_ORIENTATION:
                                    npy_array_org_left.append(npy_org_left)
                                    dup =1
                                    for dup in range(1,dupRatio):
                                        npy_array_org_left.append(npy_org_left)
                                
                                
                            elif frameRate == '24' and fcount % 4 == 0:
                                #npy dup
                                npy_array.append(npy)
                                dupcount =dupcount+1
                                dup=1
                                for dup in range(1,dupRatio):
                                    npy_array.append(npy)
                                    dupcount=dupcount+1

                                if orientation == LEFT_HAND_ORIENTATION:
                                    npy_array_org_left.append(npy_org_left)
                                    dup=1
                                    for dup in range(1,dupRatio):
                                        npy_array_org_left.append(npy_org_left)
                                
                        #for srtaring ending frame of a trial
                       
                        #check for zero padding needed
                                               
                        fcount =fcount+dupcount
                        #print('count: ',count)
                        count=0
                        for count in range(fcount,MAX_FRAME):
                            npy =np.zeros(543*3)
                            npy_array.append(npy)
                            
                        
                        if orientation == LEFT_HAND_ORIENTATION:
                            count=0
                            for count in range(fcount,MAX_FRAME):
                                npy2 =np.zeros(543*3)
                                npy_array_org_left.append(npy2)
                        #print('end count: ',count)    
                        #now class index
                        #class_array.append(classIndex);
                        #print(len(npy_array))
                        dest_class_path=f'{dest_path}/{str(classIndex)}'
                        if os.path.exists(dest_class_path):
                            pass
                        else:
                            os.makedirs(dest_class_path)
                        
                        dest_file_path =f'{dest_class_path}/{classIndex}_{user}_{trial}_{fileName}.npy'                                            
                        np.save(dest_file_path,my_flatten(npy_array))

                        if orientation == LEFT_HAND_ORIENTATION: 
                            #print(len(npy_array_org_left))
                            dest_class_path_org_left=f'{dest_path_org_left}/{str(classIndex)}'
                            if os.path.exists(dest_class_path_org_left):
                                pass
                            else:
                                os.makedirs(dest_class_path_org_left)
                            
                            dest_file_path_org_left =f'{dest_class_path_org_left}/{classIndex}_{user}_{trial}_{fileName}.npy'                        
                            np.save(dest_file_path_org_left,my_flatten(npy_array_org_left))


                        

                        
                            


                            

                            
                            
                            




In [ ]:
cv2.destroyAllWindows()


In [3]:
import os
import numpy as np


def normalizeData (norm_path, crossValidationDataPaths,testPaths):
    print("hello")
    PER_FRAME_FEATURE =1629
    min=np.zeros (PER_FRAME_FEATURE)
    max=np.zeros (PER_FRAME_FEATURE)
    

    for i in range (0,PER_FRAME_FEATURE):
        min[i]= 9
        max[i]= -9

    ml_instances_paths=[]
    for path in crossValidationDataPaths:   
        print('training path: ',path) 
        classFolders= os.listdir(path)
        for classFolder in classFolders:
            classFolder_path=f'{path}/{classFolder}'
            classTrials =os.listdir(classFolder_path)
            for trial in classTrials:
                trialPath =f'{classFolder_path}/{trial}'
                ml_instances_paths.append(trialPath)           
                npy =np.load(trialPath)            
                npy_matrix=npy.reshape(-1,PER_FRAME_FEATURE)
                feature =0
                for feature in range(0,PER_FRAME_FEATURE):
                    temp_min = np.min(npy_matrix[:,feature])
                    temp_max = np.max(npy_matrix[:,feature])
                
                    if temp_max>max[feature]:
                        max[feature] =temp_max
                    if temp_min < min[feature]:
                        min[feature] =temp_min
    
    test_paths=[]
    for path in testPaths:    
        print('test path: ',path) 
        classFolders= os.listdir(path)
        for classFolder in classFolders:
            classFolder_path=f'{path}/{classFolder}'
            classTrials =os.listdir(classFolder_path)
            for trial in classTrials:
                trialPath =f'{classFolder_path}/{trial}'            
                test_paths.append(trialPath)
                #generate min-max
                npy =np.load(trialPath)
                npy_matrix=npy.reshape(-1,PER_FRAME_FEATURE)          
            
                feature=0
                for feature in range(0,PER_FRAME_FEATURE):
                    temp_min = np.min(npy_matrix[:,feature])
                    temp_max = np.max(npy_matrix[:,feature])
                
                    if temp_max>max[feature]:
                        max[feature] =temp_max
                    if temp_min < min[feature]:
                        min[feature] =temp_min

    print('Normalization starting')

    for ml_instances_path in ml_instances_paths:         
        npy =np.load(ml_instances_path)
        npy_matrix=npy.reshape(-1,PER_FRAME_FEATURE)
        feature=0
        for feature in range(0,PER_FRAME_FEATURE):          
            my_range = max[feature]-min[feature]        
            npy_matrix[:,feature] = (npy_matrix[:,feature]*100000 -min[feature]*100000)/(my_range*100000+0.00001)     
        
        #save in file
        # toDest_path=f'{norm_path}/{ml_instances_path}'
    
        #create the target folder
        label = ml_instances_path.split('/')
        dir_path =''
        i=0
        for i in range(1,len(label)-1): #omit ../
            dir_path=dir_path+label[i]+'/'

        dir_path=f'{norm_path}/{dir_path}'
        if os.path.exists(dir_path):
            pass           
                           
        else: 
            os.makedirs(dir_path)    

        toDest_path =f'{dir_path}/{label[len(label)-1]}'
        np.save(toDest_path,npy_matrix.flatten())

    for ml_instances_path in test_paths:  
        npy =np.load(ml_instances_path)
        npy_matrix=npy.reshape(-1,PER_FRAME_FEATURE)
        feature =0
        for feature in range(0,PER_FRAME_FEATURE):
            my_range =max[feature]-min[feature]
            npy_matrix[:,feature] = (npy_matrix[:,feature]*100000 -min[feature]*100000)/(my_range*100000+0.00001)  

        #save in file
        # toDest_path=f'{norm_path}/{ml_instances_path}'

        label = ml_instances_path.split('/')
        dir_path =''
        i=0
        for i in range(1,len(label)-1):  #omit ../
            dir_path=dir_path+label[i]+'/'

        dir_path=f'{norm_path}/{dir_path}'
        if os.path.exists(dir_path):
            pass                                                   
        else: 
            os.makedirs(dir_path)
        
        toDest_path =f'{dir_path}/{label[len(label)-1]}'
        np.save(toDest_path,npy_matrix.flatten())

    



In [5]:
#NORMALIZE AS NEEDED
#normalize right hand and original lefthand data (without flip) together-----------------------------------------
norm_path ='../NORMALIZED_ORIGINAL'
crossValidationDataPaths=['../NumericalData/ML/Training/RightHand','../NumericalData/ML/Training/LeftHand_NO_FLIP']
testPaths=['../NumericalData/ML/Test/RightHand','../NumericalData/ML/Test/LeftHand_NO_FLIP']
print('normalize')
normalizeData(norm_path,crossValidationDataPaths,testPaths)
# DO THE SAME ON THE PROLONGED DATA NEEDED FOR SVM EXPERIMENTING ON ORIGINAL DATA (NO FLIP)
norm_path ='../NORMALIZED_ORIGINAL'
crossValidationDataPaths=['../NumericalData/ML_PROLONG/Training/RightHand','../NumericalData/ML_PROLONG/Training/LeftHand_NO_FLIP']
testPaths=['../NumericalData/ML_PROLONG/Test/RightHand','../NumericalData/ML_PROLONG/Test/LeftHand_NO_FLIP']
print('normalize')
normalizeData(norm_path,crossValidationDataPaths,testPaths)

normalize
hello
training path:  ../NumericalData/ML_PROLONG/Training/RightHand
training path:  ../NumericalData/ML_PROLONG/Training/LeftHand_NO_FLIP
test path:  ../NumericalData/ML_PROLONG/Test/RightHand
test path:  ../NumericalData/ML_PROLONG/Test/LeftHand_NO_FLIP
Normalization starting


In [25]:




#normalize right hand and lefthand flipped data together-----------------------------------------
norm_path ='../NORMALIZED'
crossValidationDataPaths=['../NumericalData/ML/Training/RightHand','../NumericalData/ML/Training/LeftHand']
testPaths=['../NumericalData/ML/Test/RightHand','../NumericalData/ML/Test/LeftHand']
normalizeData(norm_path,crossValidationDataPaths,testPaths)


#normalize left hand (not flipped) data if someone want to experiment with the left hand original data only
norm_path ='../NORMALIZED'
crossValidationDataPaths=['../NumericalData/ML/Training/LeftHand_NO_FLIP']
testPaths=['../NumericalData/ML/Test/LeftHand_NO_FLIP']
normalizeData(norm_path,crossValidationDataPaths,testPaths)


#normalize the prolonged data  --------------------------

norm_path ='../NORMALIZED'
crossValidationDataPaths=['../NumericalData/ML_PROLONG/Training/RightHand','../NumericalData/ML_PROLONG/Training/LeftHand']
testPaths=['../NumericalData/ML_PROLONG/Test/RightHand','../NumericalData/ML_PROLONG/Test/LeftHand']
normalizeData(norm_path,crossValidationDataPaths,testPaths)

#normalize prolonged lefthand (not flipped) orignal data in case someone want to work with lefthand original data only------
norm_path ='../NORMALIZED'
crossValidationDataPaths=['../NumericalData/ML_PROLONG/Training/LeftHand_NO_FLIP']
testPaths=['../NumericalData/ML_PROLONG/Test/LeftHand_NO_FLIP']
normalizeData(norm_path,crossValidationDataPaths,testPaths)








hello
training path:  ../NumericalData/ML_PROLONG/Training/RightHand
training path:  ../NumericalData/ML_PROLONG/Training/LeftHand
test path:  ../NumericalData/ML_PROLONG/Test/RightHand
test path:  ../NumericalData/ML_PROLONG/Test/LeftHand
Normalization starting
hello
training path:  ../NumericalData/ML_PROLONG/Training/LeftHand_NO_FLIP
test path:  ../NumericalData/ML_PROLONG/Test/LeftHand_NO_FLIP
Normalization starting


In [6]:
#DTW DISTANCES normalize right hand and lefthand flipped data together-----------------------------------------

DTW_weights_available =False
# DTW_weights_available =True

if DTW_weights_available == False:
    print ('KAGGLE DATASET NAME: DTW_BdSLW60')
    print ('WARNING: DOWNLOAD THE PRECALCULATED DTW DISTANCE DATASET FROM KAGGLE FIRST')
    print ('COPY THE DOWNLOADED DTW FOLDER IN THE PARENT DIRECTORY. YOU MAY SEE AND EMPTY DTW FOLDER THERE, OVERWRITE')
    
    print ('set DTW_weights_available flag to True')
else :
    norm_path ='../DTW_NORMALIZED'
    crossValidationDataPaths=['../DTW/Training/RightHand','../DTW/Training/LeftHand']
    testPaths=['../DTW/Test/RightHand','../DTW/Test/LeftHand']
    normalizeData(norm_path,crossValidationDataPaths,testPaths)

hello
training path:  ../DTW/Training/RightHand
training path:  ../DTW/Training/LeftHand
test path:  ../DTW/Test/RightHand
test path:  ../DTW/Test/LeftHand
Normalization starting
